# DATA CLEANING

In [1]:
import pandas as pd
import os
import glob
from pathlib import Path
import openpyxl as op
from openpyxl import load_workbook
import xlrd
from datetime import datetime
import numpy as np

## Shared functions

In [ ]:
#Excel to CSV conversion
def read_files(dir_path, store_path):
    os.chdir(dir_path)
    file_names = [i for i in glob.glob('*.{}'.format('xlsx'))]   
    for f in file_names:
        read_file = pd.read_excel (dir_path+f)        
        read_file.to_csv(store_path+Path(f).stem+'.csv', index = None, header=True)

In [ ]:
#Retrieve data from DB
def retrieve_data(db_name, username, db_host, db_password, db_port, query, column):
    conn = psycopg2.connect(dbname=db_name, user=username, host=db_host, password=db_password, port=db_port)
    cur = conn.cursor(column, cursor_factory=psycopg2.extras.DictCursor)    
    cur.execute(query)
    df = cur.fetchall()
    return df                                                    

__add_calculated_column(df)__

Check which measurement unit was already present in the file (µg/m3 or ppb) and add a column for the one not present, calculating the values with the correct formula.

FORMULAS
- molecular_weight x ppb_concentration = 24,45 x µg/m3_concentration
- µg_m3_concentration = (peso molecular_weight x ppb_concentration) ÷ 24,45
- ppb_concentration = (24,45 x µg_m3_concentration/m3) ÷ molecular_weight 

ASSUMPTIONS
- molecular_weight [g/mol]
- 24,45 is the volume (l) of a mole of gas when the temperature is 25°C and the pression in 1 atm(1 atm = 1,01325 bar).

ONLINE CONVERTER TO CHECK RESULTS
https://www.wkcgroup.com/tools-room/micrograms-per-cubic-meter-parts-per-billion-converter/ 


In [ ]:
def add_calculated_column(df):
    dic_mol_weights = {'TRS':122.14, 'C6H6':78.11, 'H2S' =34.1}
    column_names = ['TRS_ug/m3','C6H6_ug/m3', 'H2S_ug/m3']
    
    for column in column_names:         
        if (column in list(df.columns)):   
            chemical = column.replace('_ug/m3','')
            df.insert(loc=df.columns.get_loc('column'), column=column.replace('_ug/m3', '_ppb'), value=((24.45*df[column])/dic_mol_weights[chemical]))    
        else:
            chemical = column.replace('_ppb','')
            df.insert(loc=df.columns.get_loc(column)+1, column=column.replace('_ppb', '_ug/m3'), value=((dic_mol_weights[chemical]*df[column])/24.45))
    return df

**add_id_column(df, file_name)**

Add id column related to the control unit, based on the file name (the file name contains the control unit name).

- ATM05 --> id = 1
- ATM06 --> id = 2
- ATM07 --> id = 3
- ATM10 --> id = 4
- ATM11 --> id = 5
- ATM12 --> id = 6
- ATM13 --> id = 7
- ATM14 --> id = 8

In [13]:
def add_id_column(df, file_name):  
    id_dic = {'ATM05':1, 'ATM06':2, 'ATM07':3, 'ATM10':4, 'ATM11':5, 'ATM12':6,'ATM13':7, 'ATM14':8}
    for atm in dic.keys():
        if atm in file_name: 
            df.insert(0, column='atm_id', value=(id_dic[atm]))    

## Data cleaning - 2021 dataset

**merge_date_time_2021(df)**

Merging date and time in the same column for 2021 data.

Pandas reads time as float because the corresponding column was stored with '.' instead of ':'.
Perform the following steps:
- number to string conversion
- adding 0 at the beginning or end hour, e.g.:
    - 0.15 --> 00.15, 
    - 10.5 --> 10.50, 
    - 1.1 --> 01.10
- merging date and time in the same column
- removing time column
- replacing '.' with ':' for string to datetime conversion
- string to datetime conversion



In [ ]:
def merge_date_time_2021(df):
    column_names = list(df.columns)    
    if 'Data' in str(column_names[0]) and 'ora' in str(column_names[1]):                                               
        for hour in df['ora']:
            str_hour = str(hour)
            if(len(str_hour) is 4 and str_hour[1] is '.'):                 
                new_hour = str_hour.zfill(5)    
                df['ora'] = df['ora'].replace(hour,new_hour)
            elif (len(str_hour) is 4 and str_hour[2] is '.'):
                new_hour = str_hour+'0'  
                df['ora'] = df['ora'].replace(hour,new_hour)
            elif (len(str_hour) is 3 and str_hour[1] is '.'):
                new_hour = str_hour.zfill(4) + '0'
                df['ora'] = df['ora'].replace(hour,new_hour)          
        df["Data"] = (df["Data"].apply(str)+' '+df["ora"].apply(str))                     
        df.drop('ora', axis=1, inplace = True)   
        df["Data"] = df["Data"].apply(lambda x: x.replace(".", ":"))                
        df['Data'] =  pd.to_datetime(df['Data'], infer_datetime_format=True)
    return df

In [ ]:
#Delete all the unnecessary values in the 'Data' column (there are 'Data' values that don't have 
#corresponding chemical values).
def drop_date_values(df):
    column_names = list(df.columns) 
    row_index = df[df['ora'].isnull()].index.tolist()   
    for column in column_names:
        if(len(row_index) != 0 and column not in 'Data' and column not in 'ora'):            
            df = df.iloc[:row_index[0]]
    return df

In [ ]:
def clean_tipology1(df):
    df = pd.read_excel(f, sheet_name=1)            
    df.drop('postazione', axis=1, inplace = True)            
    df = drop_date_values(df)
    df = merge_date_time_2021(df) 
    df.set_axis(['data', 'trs_ppb', 'trs_stato', 'voc_ppm', 'voc_stato', 'c6h6_ug_m3', 'c6h6_stato', 'h2s_ug_m3', 'h2s_stato', 'H2SJ_ug/m3', 'H2SJ_stato','pidvoc_ppb', 'pidvoc_stato'], axis=1, inplace=True)                
    column_list = [sheet.cell(0,column).value for column in range(sheet.ncols)]            
    if 'H2SJ_ug/m3' in column_list:                
        df.drop('H2SJ_ug/m3', axis=1, inplace = True)
        df.drop('H2SJ_stato', axis=1, inplace = True)      
    else:     
        df.drop('H2SJ_ppb', axis=1, inplace = True)
        df.drop('H2SJ_stato', axis=1, inplace = True)  
    df = add_calculated_column(df) 
    add_id_columns(df, f)
    #df.to_excel(store_path+Path(f).stem+".xls", index=False)        
    df.to_csv(store_path+Path(f).stem+'.csv', index = None, header=True)
    return df

In [ ]:
def clean_tipology2(df):
    df = pd.read_excel(f, sheet_name=1, headers=0, skiprows=5)   
    df = drop_date_values(df)
    df['Data'] = df['Data'].fillna(method='ffill')
    df = merge_date_time_2021(df)                      
    column_list = [sheet.cell(5,column).value for column in range(sheet.ncols)]            
    if 'ug/m3' in column_list:   
        df.set_axis(['data', 'trs_ppb', 'trs_stato', 'voc_ppm', 'voc_stato', 'c6h6_ug_m3', 'c6h6_stato', 'h2s_ug_m3', 'h2s_stato', 'H2SJ_ug/m3', 'H2SJ_stato','pidvoc_ppb', 'pidvoc_stato'], axis=1, inplace=True)                
        df.drop('H2SJ_ug/m3', axis=1, inplace = True)
        df.drop('H2SJ_stato', axis=1, inplace = True)                       
    else:                
        df.set_axis(['data', 'trs_ppb', 'trs_stato', 'voc_ppm', 'voc_stato', 'c6h6_ppb', 'c6h6_stato', 'h2s_ppb', 'h2s_stato', 'H2SJ_ppb', 'H2SJ_stato', 'pidvoc_ppb', 'pidvoc_stato'], axis=1, inplace=True)                            
        df.drop('H2SJ_ppb', axis=1, inplace = True)
        df.drop('H2SJ_stato', axis=1, inplace = True)                    
    df = add_calculated_column(df)   
    add_id_columns(df, f)
    #df.to_excel(store_path+Path(f).stem+".xls", index=False)                     
    df.to_csv(store_path+Path(f).stem+'.csv', index = None, header=True)
    return df

__clean_2021_data(dir_path, store_path)__

_xlrd_ necessary because 2021 files are in _xls_ format.

The only sheet needed is _'Report settimanale'_.

There are two types of files; to dynamically check which one we are using, we check 
if the first row has the word _"Date"_ in it, which is not present in the other type.

Steps performed according to type:
 - Type 1: Delete the "postazione" column because it is not needed.
 - Type 2:
    * Ignored the first 5 rows (unnecessary information).
    * added the missing values for the date column.
    * added header because some columns are unnamed due to bad reading of the file. 2 types of header because in some cases there were errors
      in units _ppb_ instead of _ug/m3_ and vice versa
    
Shared steps (type 1 and 2):
  -  *merge_date_time_2021*
  - deleted columns for _H2SJ_ because these sensors were broken, so data are incorrect
  -  *add_calculated_column*
  -  *add_id_column*
  - converted file to .csv 


In [ ]:
def clean_2021_data(dir_path, store_path):    
    os.chdir(os.path.abspath(dir_path))    
    file_names = [i for i in glob.glob('*.{}'.format('xls'))]       
    for f in file_names:        
        wb = xlrd.open_workbook(f);
        sheet = wb.sheet_by_name('Report settimanale')           
        df = pd.DataFrame()         
        if "Data" in sheet.cell(0, 1).value:                
            dff = clean_tipology1(df)
        else:            
            dff = clean_tipology2(df)
    return dff  

In [ ]:
dff = clean_2021_data("../DATASET_2021/ATM05/", "../CLEANED_DATA_2021/ATM05/")

## Data cleaning - 2022 dataset

**merge_date_time_2022(df)**

Merging date and time in the same column for 2022 data.

_'data'_ and _'ora'_ were already in the correct format, the only two steps performed were:
merging date and time in the same column
- removing time column
- string to datetime conversion

In [ ]:
def merge_date_time_2022(df):
    column_names = list(df.columns)     
    if 'data' in str(column_names[0]) and 'ora' in str(column_names[1]):                                                               
        df["data"] = (df["data"].apply(str)+' '+df["ora"].apply(str))                     
        df.drop('ora', axis=1, inplace = True)                      
        df['data'] =  pd.to_datetime(df['data'], infer_datetime_format=True)
    return df

__clean_2022_data(dir_path, store_path)__

Steps performed:
- *merge_date_time_2022*
- deleted columns for _H2SJ_ because these sensors were broken, so data are incorrect
- conversion of columns containing numeric values to _float_ because their type was originally _object_ and therefore it was not possible to perform the mathematical operations required by the _add_calculated_column_ function
- changed the header to match that of 2021
- replacedstrings *** used in place of empty cells with NaN 
- _add_calculated_column_
- _add_id_column_
- converted to .csv 




In [ ]:
def clean_2022_data(dir_path, store_path):
    os.chdir(os.path.abspath(dir_path))    
    file_names = [i for i in glob.glob('*.{}'.format('csv'))]     
    df = pd.DataFrame() 
    for f in file_names:               
        df = pd.read_csv(f, sep='\t')             
        df.drop('H2SJ', axis=1, inplace = True)
        df.drop('Status.4', axis=1, inplace = True)
        df = df.replace('***', np.nan)
        column_names = list(df.columns)
        for column in column_names:
            if 'Date' not in column and 'Ora' not in column and 'Status' not in column:
                df[column] = df[column].astype(float)               
        df.set_axis(['data', 'ora', 'trs_ppb', 'trs_stato', 'voc_ppm', 'voc_stato', 'c6h6_ug_m3', 'c6h6_stato', 'h2s_ug_m3', 'h2s_stato','pidvoc_ppb', 'pidvoc_stato', 'Unnamed'], axis=1, inplace=True)
        df.drop('Unnamed', axis=1, inplace=True)                         
        df = merge_date_time_2022(df)                
        df = add_calculated_column(df) 
        add_id_column(df, f)
        df.to_csv(store_path+Path(f).stem+'.csv', index = None, header=True)
    return df 

In [ ]:
dff = clean_2022_data("../DATASET_2022/ATM05/", "../CLEANED_DATA_2022/ATM05/")

## Data cleaning - meteo dataset 2022

**clean_meteo_data_22(dir_path, store_path)**

Steps performed:

- chaged the headers to improve readability (e.g., winddirection --> wind_direction)
- replaced the 'T' character present in the dates with a blank space so that the format is similar to that of the other data

In [ ]:
def clean_meteo_data_22(dir_path, store_path):
    os.chdir(os.path.abspath(dir_path))    
    file_names = [i for i in glob.glob('*.{}'.format('csv'))]     
    df = pd.DataFrame() 
    for f in file_names:                       
        df = pd.read_csv(f, sep=',') 
        df.set_axis(['date', 'wind_speed', 'wind_direction', 'temperature', 'radiation', 'pressure', 'precipitation', 'humidity'], axis=1, inplace=True)
        #df = clean_meteo_date_22(df)
        column_names = list(df.columns)     
        if 'date' in str(column_names[0]):        
            df["date"] = df["date"].apply(lambda x: x.replace("T", " "))
            df.to_csv(store_path+Path(f).stem+'.csv', index = None, header=True)
    return df

In [ ]:
dff = clean_meteo_data_22("../METEO_DATA/2022_METEO_DATA/", "../CLEANED_METEO_DATA/CLEANED_METEO_2022/")

## Additional cleaning

Different data formats, 'DD/MM/YYYY' in some file, 'MM/DD/YYYY' in others. 
It become a problem when there were data out of the available range.
Considering 2021 data, we have data between 04/12/2021 and 11/28/2021 in the MM/DD/YYYY format, so if there is 03/11/2021 it is considered a March data, instead in the correct format it is a November data, and March isn't in the available range at all.

In [3]:
def check_min_data(data, month, day):
    if(data.month < month or (data.month == month and data.day < day)):
        return True
    
def check_max_data(data, month, day):
    if(data.month > month or data.month == day):
        return True

def change_data_format(db_name, username, db_password, db_port, query, column, file_path):
    df = retrieve_data(db_name, username, db_password, db_port, query, column)
    df = pd.DataFrame(np.array(df), columns = ['atm_id', 'data', 'trs_ppb', 'trs_ug_m3', 'trs_stato', 'voc_ppm', 'voc_stato', 'c6h6_ppb', 'c6h6_ug_m3', 'c6h6_stato', 'h2s_ppb', 'h2s_ug_m3', 'h2s_stato', 'pidvoc_ppb', 'pidvoc_stato'])
    df['data'] = pd.to_datetime(df['data'], format = '%Y-%m-%d %H:%M:%S')           
    for elem in range(0, len(df['data'])):          
        if(df.loc[elem].at["data"].year == 2021 and (check_min_data(df.loc[elem].at["data"], 4, 12) or check_max_data(df.loc[elem].at["data"], 11, 28))):            
            date = df.loc[elem].at["data"].strftime('%Y-%d-%m %H:%M:%S')              
            df.at[elem,'data'] = date
        elif(df.loc[elem].at["data"].year == 2022 and (check_min_data(df.loc[elem].at["data"], 6, 20) or df.loc[elem].at["data"].month > 8)):                          
            date = df.loc[elem].at["data"].strftime('%Y-%d-%m %H:%M:%S')                
            df.at[elem,'data'] = date
    df = df.sort_values(by='data') 
    df.to_csv(file_path, index = None, header=True)    

In [ ]:
atm_list = ['atm05', 'atm06', 'atm07', 'atm10', 'atm11', 'atm12', 'atm13', 'atm14']
for atm in atm_list:
    query = 'SELECT * FROM ' + atm
    change_data_format(db_name, username, password, db_port, query, 'data', '../cleaned_data/cleaned_db.csv')